In [ ]:
import pandas as pd
books = pd.read_csv("../datasets/books_with_emotion.csv")
books

In [36]:
df = books[["FormattedBookID","URL"]]

In [28]:
import numpy as np
from bs4 import BeautifulSoup
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [46]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd

# Custom headers to avoid being blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to scrape the book image URL
def scrape_images(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            og_image_tag = soup.find('meta', {'property': 'og:image'})
            if og_image_tag:
                return og_image_tag['content']
        return "Not Found"
    except requests.exceptions.RequestException:
        return "Request Failed"

In [ ]:
url = "https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone"
scrape_images(url)

In [ ]:
# Number of threads for parallel processing
MAX_THREADS = 20  
# Progress bar setup
tqdm.pandas()

# Start timing
start_time = time.time()

# Run scraping in parallel
image_results = []
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    future_to_url = {executor.submit(scrape_images, url): url for url in df["URL"]}
    
    for future in tqdm(as_completed(future_to_url), total=len(df), desc="Scraping Images"):
        url = future_to_url[future]
        try:
            image_url = future.result()
        except Exception:
            image_url = "Error"
        image_results.append((url, image_url))

# Convert results to a dictionary and update the DataFrame
image_dict = dict(image_results)
df["Img_Path"] = df["URL"].map(image_dict)

# End timing
end_time = time.time()
total_time = end_time - start_time

print(f"\n✅ Scraped {len(df)} books in {total_time:.2f} seconds using {MAX_THREADS} threads!")
print(df.head())  # Show the first few rows

In [49]:
df.to_csv("Books_Img_Path",index=False)

In [ ]:
df["Img_Path"][0]

In [ ]:
df

In [ ]:
books = pd.merge(df[["FormattedBookID","Img_Path"]],books,on="FormattedBookID")

In [ ]:
books.to_csv("books_with_image_paths.csv",index=False)